In [141]:
# set up imports
import pandas as pd, numpy as np, seaborn as sns
from tabulate import tabulate
import statsmodels
import arch
import matplotlib
matplotlib.use('qt5agg')

# configure plot style
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams["figure.figsize"] = (9.5,4.15)
plt.rcParams['figure.constrained_layout.use'] = False
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['lines.linewidth'] = 0.8
save_plot_to =  r'C:\\Users\joche\OneDrive\03 TUM - TUM-BWL\Semester 8\01 Bachelorarbeit\04 Results\Plots/'

In [142]:
# load data: new_btc.txt, created with M4
index_name = 'date'
btc = pd.read_csv('Data/btc_new.txt', index_col=0)
btc.index = pd.DatetimeIndex(btc.index, normalize=True).normalize()
btc

In [143]:
# split data in pre and post bubble 2018
pre_bubble_end_date = '2017-12-01'
post_bubble_start_date = '2018-02-01'

# pre bubble df
btc_pre = btc[:pre_bubble_end_date]

# post bubble df
btc_post = btc[post_bubble_start_date:]

In [165]:
#define overview-printing function of dataframe
def dates_overview(dataframe):
    print('start_date:\t', dataframe.index[0])
    print('end_date:\t', dataframe.index[-1])
    print('len df:\t\t', len(dataframe))

# define train/test-splitting function of dataframe
def split_traintest_df(dataframe):
    train_size = int(len(dataframe) * 0.8)
    df_train, df_test = dataframe[0:train_size], dataframe[train_size:]
    # create dataframe for printout
    data = {'Dataframe': ['dataframe', 'df_train', 'df_test'],
            'date_start': [dataframe.index[0], df_train.index[0], df_test.index[0]],
            'date_end': [dataframe.index[-1], df_train.index[-1], df_test.index[-1]],
            'nobs': [len(dataframe), len(df_train), len(df_test)]}
    df_print = pd.DataFrame(data, columns=['Dataframe','date_start','date_end','nobs'])
    print(df_print)
    return df_train, df_test

# define log-taking and relabeling function
def log_of_df(dataframe):
    df_log = np.log(dataframe)
    new_cols = list()
    for i in df_log.columns:
        new_cols.append(i+'_log')
    df_log.columns = new_cols
    # fill na value of negative oil price on 2020-04-20 with 0
    df_log.fillna(value=0, inplace=True)
    return df_log

# define first difference-taking function of dataframe
def diff_of_df(dataframe):
    df_train_log_diff = dataframe.diff()
    # relabel columns
    new_cols = list()
    for i in df_train_log_diff.columns:
        new_cols.append(i+'_diff')
    df_train_log_diff.columns = new_cols
    return df_train_log_diff

# define return creating function of dataframe
def create_returns(dataframe_train):
    dataframe_train_log = log_of_df(dataframe_train)
    dataframe_returns = diff_of_df(dataframe_train_log)
    dataframe_returns.columns = ['btc_ret']
    dataframe_returns_squared = dataframe_returns**2
    dataframe_returns_squared.columns = ['btc_ret_sqrd']
    return dataframe_returns, dataframe_returns_squared

# define log and ret + ACF & PACF plotting function for dataframe_log, dataframe_ret
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
def plot_acf_pacf(dataframe_ret, dataframe_ret_sqrd):
    fig, axs = plt.subplots(2,2)
    axs[0][0].plot(dataframe_ret.iloc[1:], color='black')
    axs[0][0].set_title('btc_ret')
    axs[0][1].plot(dataframe_ret_sqrd.iloc[1:], color='black')
    axs[0][1].set_title('btc_ret_sqrd')
    plot_acf(dataframe_ret.iloc[1:], color='black', ax=axs[1][0], markersize=2)
    plot_acf(dataframe_ret_sqrd.iloc[1:], color='black', ax=axs[1][1], markersize=2)
    fig.set_size_inches(12,5)
    fig.tight_layout()
    return fig

# define ARCH fitting model for a dataframe and print summary to stdout/latex
import warnings
warnings.filterwarnings('ignore')
from arch import arch_model
def get_arch_model_summary(dataframe_ret, part, p_list, mean, dist , latex):
    for i in p_list:
        model = arch_model(dataframe_ret.iloc[1:], p=i,q=0, vol='ARCH', mean=mean, dist=dist)
        model_fitted = model.fit(disp=False)
        if latex is False:
            # write summary as plain text to std.out
            print('{0}: ARCH({1}):\n'.format(part,i),
            model_fitted.summary(),'\n')
        else:
            # write summary as latex to file
            with open(save_plot_to + 'M3_{0}_ARCH_configuratios_LaTeX.txt'.format(part), 'a') as myfile:
                    myfile.write('{0}: ARCH({1}):\n'.format(part,i)+
                                 model_fitted.summary().as_latex()+'\n\n')
                
# define GARCH fitting model for a dataframe and print summary to stdout/latex
import warnings
warnings.filterwarnings('ignore')
from arch import arch_model
def get_garch_model_summary(dataframe_ret, part, mean, dist, latex):
    model = arch_model(dataframe_ret.iloc[1:], p=1, q=1, mean=mean, dist=dist)
    model_fitted = model.fit(disp=False)
    if latex is False:
        # write summary as plain text to std.out
        print('{0}: GARCH(1,1):\n'.format(part),
        model_fitted.summary(),'\n')
    else:
        # write summary as latex to file
        with open(save_plot_to + 'M3_{0}_GARCH_configuratios_LaTeX.txt'.format(part), 'a') as myfile:
                myfile.write('{0}: GARCH(1,1):\n'.format(part)+
                            model_fitted.summary().as_latex()+'\n\n')

# define a GARCH(1,1) model fitting and forecast producing function for dataframe
from sklearn.metrics import mean_squared_error as mse
def get_garch_forecast(dataframe):
    # split dataframe_ret in train and test data
    train_size = int(len(dataframe) * 0.8)
    # obtain dataframe returns
    dataframe_log = log_of_df(dataframe)
    dataframe_returns = diff_of_df(dataframe_log)
    dataframe_returns.columns = ['btc_ret']
    # model garch with dataframe_ret, fit on dataframe_train and predict for dataframe_test
    model = arch_model(dataframe_returns[1:], p=1, q=1, mean='zero', dist='t')
    model_fitted = model.fit(last_obs=dataframe_returns.index[train_size-1], disp=False)
    model_result = model_fitted.forecast(horizon=1, align='target')
    # obtain variance of dataframe_ret: rolling window = 15, center=True, shift = 1
    dataframe_var = dataframe_returns.rolling(15, center=True).std().shift()**2
    # prepare for calculating mse
    dataframe_var_test = dataframe_var[train_size:].fillna(0)
    dataframe_var_predicted = model_result.variance.iloc[train_size-1:].fillna(0)
    # return [0]: mse, [1]: forecasted_var, [2]: dataframe_var
    return np.sqrt(mse(dataframe_var_predicted.iloc[:,0].values, dataframe_var_test.iloc[:,0].values)), model_result.variance.iloc[train_size-1:], dataframe_var

# define series-generating and plotting function for btc_garch_forecast
def plot_garch_forecast(dataframe_garch_forecast, dataframe_variance):
    # make series for plotting pred. vs. actual
    train_size = int(len(dataframe_variance) * 0.8)
    dataframe_train, dataframe_test = dataframe_variance[0:train_size], dataframe_variance[train_size:]
    index_pred = np.arange(len(dataframe_train) + 1, len(dataframe_variance) + 1)
    pred_variance_series = pd.Series(dataframe_garch_forecast.iloc[:,0].values, index=index_pred)
    test_variance_series = pd.Series(dataframe_test.iloc[:,0].values, index=index_pred)
    # create plot
    fig, axs = plt.subplots(1,2)
    axs[0].plot(dataframe_train.values, label= r'$\sigma^2_{btc,T}$', color='black')
    axs[0].plot(test_variance_series, label= r'$\sigma^2_{btc,T+h}$', color='green')
    axs[0].plot(pred_variance_series, label= r'$\hat{\sigma}^2_{btc,T+h}$', color= 'red')
    axs[0].legend(loc='upper left')
    axs[0].title.set_text('Gesamter Zeitraum')
    axs[1].plot(test_variance_series, label= r'$\sigma^2_{btc,T+h}$', color='green')
    axs[1].plot(pred_variance_series, label= r'$\hat{\sigma}^2_{btc,T+h}$', color= 'red')
    axs[1].legend(loc='upper left')
    axs[1].title.set_text('Vorhersage-Zeitraum')
    fig.set_size_inches(15,2.5)
    return fig

In [6]:
# overview for btc
dates_overview(btc)

In [148]:
# overview for btc_pre
dates_overview(btc_pre)

start_date:	 2015-08-07 00:00:00
end_date:	 2017-12-01 00:00:00
len df:		 533


In [149]:
# overview for btc_post
dates_overview(btc_post)

start_date:	 2018-02-01 00:00:00
end_date:	 2020-06-24 00:00:00
len df:		 540


In [6]:
# split btc in train and test dataset: proceed only with btc_train
btc_train, btc_test = split_traintest_df(btc)

In [150]:
# split btc_pre in train and test dataset: proceed only with btc_pre_train
btc_pre_train, btc_pre_test = split_traintest_df(btc_pre)

   Dataframe date_start   date_end  nobs
0  dataframe 2015-08-07 2017-12-01   533
1   df_train 2015-08-07 2017-06-16   426
2    df_test 2017-06-19 2017-12-01   107


In [151]:
# split btc_post in train and test dataset: proceed only with btc_post_train
btc_post_train, btc_post_test = split_traintest_df(btc_post)

   Dataframe date_start   date_end  nobs
0  dataframe 2018-02-01 2020-06-24   540
1   df_train 2018-02-01 2019-12-31   432
2    df_test 2020-01-02 2020-06-24   108


In [40]:
# create returns and squared returns for btc_train
btc_ret, btc_ret_sqrd = create_returns(btc_train)
btc_ret_sqrd

In [152]:
# create returns and squared returns for btc_pre_train
btc_pre_ret, btc_pre_ret_sqrd = create_returns(btc_pre_train)
btc_pre_ret_sqrd

,btc_ret_sqrd
date,
2015-08-07,NaN
2015-08-10,0.003087
2015-08-11,0.000490
2015-08-12,0.000223
2015-08-13,0.000075
...,...
2017-06-12,0.003588
2017-06-13,0.000456
2017-06-14,0.006513


In [153]:
# create returns and squared returns for btc_post_train
btc_post_ret, btc_post_ret_sqrd = create_returns(btc_post_train)
btc_post_ret_sqrd

,btc_ret_sqrd
date,
2018-02-01,NaN
2018-02-02,1.425532e-03
2018-02-05,5.699693e-02
2018-02-06,1.181769e-02
2018-02-07,2.979725e-04
...,...
2019-12-23,3.524819e-04
2019-12-24,2.034109e-05
2019-12-27,1.971364e-05


In [154]:
# plot btc_ret and btc_ret_sqrd with ACF, respectively
fig_1 = plot_acf_pacf(btc_ret,btc_ret_sqrd)

In [155]:
# save fig_1
fig_1.savefig(save_plot_to+'M3_fig_1.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [163]:
# plot btc_pre_ret and btc_pre_ret_sqrd with ACF, respectively
fig_1_pre = plot_acf_pacf(btc_pre_ret,btc_pre_ret_sqrd)

In [157]:
# save fig_1_pre
fig_1_pre.savefig(save_plot_to+'M3_pre_fig_1.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [164]:
# plot btc_post_ret and btc_post_ret_sqrd with ACF, respectively
fig_1_post = plot_acf_pacf(btc_post_ret,btc_post_ret_sqrd)

In [159]:
# save fig_1_post
fig_1_post.savefig(save_plot_to+'M3_post_fig_1.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [166]:
# get ARCH configuratios for btc_ret
p_list = {1,2,4,7}
get_arch_model_summary(btc_ret, part='', p_list=p_list, mean='zero', dist='t', latex=False)

Iteration:      1,   Func. Count:      5,   Neg. LLF: 10394.071785307493
Iteration:      2,   Func. Count:     10,   Neg. LLF: -589.9669214517892
Iteration:      3,   Func. Count:     15,   Neg. LLF: -1549.0308839208208
Iteration:      4,   Func. Count:     19,   Neg. LLF: -1466.9162461713258
Iteration:      5,   Func. Count:     24,   Neg. LLF: -858.7376633324922
Iteration:      6,   Func. Count:     31,   Neg. LLF: -1100.9761214502082
Iteration:      7,   Func. Count:     36,   Neg. LLF: -1093.9525850635496
Iteration:      8,   Func. Count:     41,   Neg. LLF: -1118.4263348530499
Iteration:      9,   Func. Count:     46,   Neg. LLF: -990.1860339942577
Iteration:     10,   Func. Count:     51,   Neg. LLF: -1120.3146014108756
Iteration:     11,   Func. Count:     56,   Neg. LLF: -1558.9953274383734
Iteration:     12,   Func. Count:     61,   Neg. LLF: -1560.3381639273985
Iteration:     13,   Func. Count:     65,   Neg. LLF: -1561.7087039367202
Iteration:     14,   Func. Count:     69, 

In [167]:
# get ARCH configuratios for btc_ret as latex
p_list = {1,2,4,7}
get_arch_model_summary(btc_ret, part='', p_list=p_list, mean='zero', dist='t', latex=True)

Iteration:      1,   Func. Count:      5,   Neg. LLF: 10394.071785307493
Iteration:      2,   Func. Count:     10,   Neg. LLF: -589.9669214517892
Iteration:      3,   Func. Count:     15,   Neg. LLF: -1549.0308839208208
Iteration:      4,   Func. Count:     19,   Neg. LLF: -1466.9162461713258
Iteration:      5,   Func. Count:     24,   Neg. LLF: -858.7376633324922
Iteration:      6,   Func. Count:     31,   Neg. LLF: -1100.9761214502082
Iteration:      7,   Func. Count:     36,   Neg. LLF: -1093.9525850635496
Iteration:      8,   Func. Count:     41,   Neg. LLF: -1118.4263348530499
Iteration:      9,   Func. Count:     46,   Neg. LLF: -990.1860339942577
Iteration:     10,   Func. Count:     51,   Neg. LLF: -1120.3146014108756
Iteration:     11,   Func. Count:     56,   Neg. LLF: -1558.9953274383734
Iteration:     12,   Func. Count:     61,   Neg. LLF: -1560.3381639273985
Iteration:     13,   Func. Count:     65,   Neg. LLF: -1561.7087039367202
Iteration:     14,   Func. Count:     69, 

In [170]:
# get ARCH configurations for btc_pre_ret
p_list = {1,3,4}
get_arch_model_summary(btc_pre_ret, part='pre', p_list=p_list, mean='zero', dist='t', latex=False)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -750.9150059592318
Iteration:      2,   Func. Count:      9,   Neg. LLF: -828.8732028044528
pre: ARCH(1):
                            Zero Mean - ARCH Model Results                           
Dep. Variable:                      btc_ret   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.002
Vol Model:                             ARCH   Log-Likelihood:                828.873
Distribution:      Standardized Student's t   AIC:                          -1651.75
Method:                  Maximum Likelihood   BIC:                          -1639.59
                                              No. Observations:                  425
Date:                      Sun, Sep 27 2020   Df Residuals:                      422
Time:                              15:50:37   Df Model:                            3
                              Volatility Model                            

In [171]:
# get ARCH configurations for btc_pre_ret as latex
p_list = {1,3,4}
get_arch_model_summary(btc_pre_ret, part='pre', p_list=p_list, mean='zero', dist='t', latex=True)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -750.9150059592318
Iteration:      2,   Func. Count:      9,   Neg. LLF: -828.8732028044528
Iteration:      1,   Func. Count:      6,   Neg. LLF: -840.0706305246688
Iteration:      1,   Func. Count:      7,   Neg. LLF: -851.169641747286


In [172]:
# get ARCH configurations for btc_post_ret
p_list = {2,3}
get_arch_model_summary(btc_post_ret, part='post', p_list=p_list, mean='zero', dist='t', latex=False)

Iteration:      1,   Func. Count:      6,   Neg. LLF: -344.49604692749836
Iteration:      2,   Func. Count:     12,   Neg. LLF: 61.90128766426415
Iteration:      3,   Func. Count:     18,   Neg. LLF: -339.88351995851315
Iteration:      4,   Func. Count:     24,   Neg. LLF: -698.2370205947766
Iteration:      5,   Func. Count:     30,   Neg. LLF: -480.89784530554226
Iteration:      6,   Func. Count:     36,   Neg. LLF: -480.8038277357555
Iteration:      7,   Func. Count:     42,   Neg. LLF: -502.9187491792141
Iteration:      8,   Func. Count:     48,   Neg. LLF: -492.43870849183764
Iteration:      9,   Func. Count:     54,   Neg. LLF: -540.7433214482281
Iteration:     10,   Func. Count:     60,   Neg. LLF: -507.4658169185074
Iteration:     11,   Func. Count:     66,   Neg. LLF: -753.0746825785295
Iteration:     12,   Func. Count:     72,   Neg. LLF: -763.1265878122374
Iteration:     13,   Func. Count:     77,   Neg. LLF: -763.6778193764849
Iteration:     14,   Func. Count:     82,   Neg.

In [173]:
# get ARCH configurations for btc_post_ret as latex
p_list = {2,3}
get_arch_model_summary(btc_post_ret, part='post', p_list=p_list, mean='zero', dist='t', latex=True)

Iteration:      1,   Func. Count:      6,   Neg. LLF: -344.49604692749836
Iteration:      2,   Func. Count:     12,   Neg. LLF: 61.90128766426415
Iteration:      3,   Func. Count:     18,   Neg. LLF: -339.88351995851315
Iteration:      4,   Func. Count:     24,   Neg. LLF: -698.2370205947766
Iteration:      5,   Func. Count:     30,   Neg. LLF: -480.89784530554226
Iteration:      6,   Func. Count:     36,   Neg. LLF: -480.8038277357555
Iteration:      7,   Func. Count:     42,   Neg. LLF: -502.9187491792141
Iteration:      8,   Func. Count:     48,   Neg. LLF: -492.43870849183764
Iteration:      9,   Func. Count:     54,   Neg. LLF: -540.7433214482281
Iteration:     10,   Func. Count:     60,   Neg. LLF: -507.4658169185074
Iteration:     11,   Func. Count:     66,   Neg. LLF: -753.0746825785295
Iteration:     12,   Func. Count:     72,   Neg. LLF: -763.1265878122374
Iteration:     13,   Func. Count:     77,   Neg. LLF: -763.6778193764849
Iteration:     14,   Func. Count:     82,   Neg.

In [174]:
# get GARCH configurations for btc_ret
get_garch_model_summary(btc_ret, part='', mean='zero', dist='t', latex=False)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 10394.071785307493
Iteration:      2,   Func. Count:     13,   Neg. LLF: 262.10938129005547
Iteration:      3,   Func. Count:     19,   Neg. LLF: -1557.2429742056102
Iteration:      4,   Func. Count:     30,   Neg. LLF: -1535.8535032188763
Iteration:      5,   Func. Count:     36,   Neg. LLF: -1287.4594603519863
Iteration:      6,   Func. Count:     42,   Neg. LLF: -1610.9014414516203
: GARCH(1,1):
                           Zero Mean - GARCH Model Results                           
Dep. Variable:                      btc_ret   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.001
Vol Model:                            GARCH   Log-Likelihood:                1610.90
Distribution:      Standardized Student's t   AIC:                          -3213.80
Method:                  Maximum Likelihood   BIC:                          -3194.65
                                 

In [175]:
# get GARCH configurations for btc_ret as latex
get_garch_model_summary(btc_ret, part='', mean='zero', dist='t', latex=True)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 10394.071785307493
Iteration:      2,   Func. Count:     13,   Neg. LLF: 262.10938129005547
Iteration:      3,   Func. Count:     19,   Neg. LLF: -1557.2429742056102
Iteration:      4,   Func. Count:     30,   Neg. LLF: -1535.8535032188763
Iteration:      5,   Func. Count:     36,   Neg. LLF: -1287.4594603519863
Iteration:      6,   Func. Count:     42,   Neg. LLF: -1610.9014414516203


In [176]:
# get GARCH configurations for btc_pre_ret
get_garch_model_summary(btc_pre_ret, part='pre', mean='zero', dist='t', latex=False)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -854.8405415226746
pre: GARCH(1,1):
                           Zero Mean - GARCH Model Results                           
Dep. Variable:                      btc_ret   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.002
Vol Model:                            GARCH   Log-Likelihood:                854.841
Distribution:      Standardized Student's t   AIC:                          -1701.68
Method:                  Maximum Likelihood   BIC:                          -1685.47
                                              No. Observations:                  425
Date:                      Sun, Sep 27 2020   Df Residuals:                      421
Time:                              15:55:04   Df Model:                            4
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0%

In [177]:
# get GARCH configurations for btc_pre_ret as latex
get_garch_model_summary(btc_pre_ret, part='pre', mean='zero', dist='t', latex=True)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -854.8405415226746


In [178]:
# get GARCH configurations for btc_post_ret
get_garch_model_summary(btc_post_ret, part='post', mean='zero', dist='t', latex=False)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -760.6221048321894
post: GARCH(1,1):
                           Zero Mean - GARCH Model Results                           
Dep. Variable:                      btc_ret   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.002
Vol Model:                            GARCH   Log-Likelihood:                760.622
Distribution:      Standardized Student's t   AIC:                          -1513.24
Method:                  Maximum Likelihood   BIC:                          -1496.98
                                              No. Observations:                  431
Date:                      Sun, Sep 27 2020   Df Residuals:                      427
Time:                              15:56:33   Df Model:                            4
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0

In [179]:
# get GARCH configurations for btc_post_ret as latex
get_garch_model_summary(btc_post_ret, part='post', mean='zero', dist='t', latex=True)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -760.6221048321894


In [138]:
# forecast GARCH(1,1) with btc
btc_mse, btc_garch_forecast, btc_variance = get_garch_forecast(btc)
print('MSE:\t', btc_mse)

In [146]:
# plot GARCH(1,1) forecast with btc
fig_2 = plot_garch_forecast(btc_garch_forecast, btc_variance)

In [180]:
# save fig_2
fig_2.savefig(save_plot_to+'M3_fig_2.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [181]:
# forecast GARCH(1,1) with btc_pre
btc_pre_mse, btc_pre_garch_forecast, btc_pre_variance = get_garch_forecast(btc_pre)
print('MSE_pre:\t', btc_pre_mse)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -852.6821364490052
MSE_pre:	 0.0019413647192159083


In [182]:
# plot GARCH(1,1) forecast with btc_pre
fig_2_pre = plot_garch_forecast(btc_pre_garch_forecast, btc_pre_variance)

In [183]:
# save fig_2_pre
fig_2_pre.savefig(save_plot_to+'M3_pre_fig_2.svg',format='svg',bbox_inches='tight',pad_inches = 0)

In [184]:
# forecast GARCH(1,1) with btc_post
btc_post_mse, btc_post_garch_forecast, btc_post_variance = get_garch_forecast(btc_post)
print('MSE_post:\t', btc_post_mse)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 5128.045941903738
Iteration:      2,   Func. Count:     13,   Neg. LLF: -383.96577011114744
Iteration:      3,   Func. Count:     19,   Neg. LLF: -760.259401719677
Iteration:      4,   Func. Count:     24,   Neg. LLF: -766.2955990826204
MSE_post:	 0.004780353048633315


In [185]:
# plot GARCH(1,1) forecast with btc_post
fig_2_post = plot_garch_forecast(btc_post_garch_forecast, btc_post_variance)

In [186]:
# save fig_2_post
fig_2_post.savefig(save_plot_to+'M3_post_fig_2.svg',format='svg',bbox_inches='tight',pad_inches = 0)

